In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
from PIL import Image  # Assuming you want to work with images

# Define the path to your dataset
dataset_path = '/content/gdrive/MyDrive/AllCrops'

# List all folders in the dataset
folders = os.listdir(dataset_path)

# Iterate through each folder
for folder in folders:
    folder_path = os.path.join(dataset_path, folder)

    # List all files (images) in the folder
    files = os.listdir(folder_path)

    # Iterate through each file (image) in the folder
    for file in files:
        file_path = os.path.join(folder_path, file)

        # Do something with the file, for example, display the image

In [ ]:
IMAGE_SIZE=128
BATCH_SIZE=32
CHANNELS=3
EPOCHS=50

In [ ]:
from tensorflow import keras
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

Found 1128 files belonging to 4 classes.


In [ ]:
len(dataset)

36

In [ ]:
class_names=dataset.class_names
class_names

['Resized-Alluvial', 'Resized-Black', 'Resized-Clay', 'Resized-Red']

In [ ]:
train_size=0.8
len(dataset)*train_size

28.8

In [ ]:
train_ds=dataset.take(28)

In [ ]:
test_ds=dataset.skip(28)

In [ ]:
val_size=0.1
len(dataset)*val_size

3.6

In [ ]:
val_ds=test_ds.take(3)

In [ ]:
test_ds=test_ds.skip(3)

In [ ]:
def get_dataset_partition_tf(ds,train_split=0.8,test_split=0.1,val_split=0.1,shuffle=True,shuffle_size=1000):
  ds_size=len(ds)
  if shuffle:
    ds=ds.shuffle(shuffle_size,seed=12)
  train_size=int(train_split*ds_size)
  val_size=int(val_split*ds_size)

  train_ds=ds.take(train_size)

  val_ds=ds.skip(train_size).take(val_size)
  test_ds=ds.skip(train_size).skip(val_size)

  return train_ds,val_ds,test_ds

In [ ]:
train_ds,val_ds,test_ds=get_dataset_partition_tf(dataset)

In [ ]:
len(train_ds)

28

In [ ]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_and_rescale=tf.keras.Sequential([
    keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    keras.layers.experimental.preprocessing.Rescaling(1.0/255)
]

)

In [ ]:
data_augmentation=tf.keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
IMAGE_SIZE = 128
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

n_classes = 4

model = Sequential([
    # Assuming resize_and_rescale and data_augmentation are properly defined
    layers.InputLayer(input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax')
])


model.build(input_shape=(None, IMAGE_SIZE, IMAGE_SIZE, CHANNELS))

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 64)       

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,

    validation_data=val_ds
)

Epoch 1/50
28/28 [==============================] - 226s 150ms/step - loss: 8.0301 - accuracy: 0.5424 - val_loss: 0.8072 - val_accuracy: 0.6354
Epoch 2/50
28/28 [==============================] - 1s 19ms/step - loss: 0.5526 - accuracy: 0.7913 - val_loss: 0.3152 - val_accuracy: 0.9271
Epoch 3/50
28/28 [==============================] - 0s 16ms/step - loss: 0.3554 - accuracy: 0.8560 - val_loss: 0.2538 - val_accuracy: 0.9688
Epoch 4/50
28/28 [==============================] - 0s 18ms/step - loss: 0.3111 - accuracy: 0.8873 - val_loss: 0.2837 - val_accuracy: 0.8958
Epoch 5/50
28/28 [==============================] - 0s 17ms/step - loss: 0.3609 - accuracy: 0.8616 - val_loss: 0.1927 - val_accuracy: 0.9583
Epoch 6/50
28/28 [==============================] - 0s 16ms/step - loss: 0.2490 - accuracy: 0.9107 - val_loss: 0.1380 - val_accuracy: 0.9583
Epoch 7/50
28/28 [==============================] - 0s 16ms/step - loss: 0.1631 - accuracy: 0.9453 - val_loss: 0.1592 - val_accuracy: 0.9583
Epoch 8/50

In [ ]:
dataset_path_test = '/content/gdrive/MyDrive/AllCrops-test'

In [ ]:
score=model.evaluate(test_ds)

5/5 [==============================] - 3s 8ms/step - loss: 0.1563 - accuracy: 0.9625


In [ ]:
# List all folders in the dataset
folders = os.listdir(dataset_path_test)

# Iterate through each folder
for folder in folders:
    folder_path = os.path.join(dataset_path_test, folder)

    # List all files (images) in the folder
    files = os.listdir(folder_path)

    # Iterate through each file (image) in the folder
    for file in files:
        file_path = os.path.join(folder_path, file)

In [ ]:
IMAGE_SIZE=128
BATCH_SIZE=16
CHANNELS=3
EPOCHS=50

In [ ]:
from tensorflow import keras
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path_test,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

Found 338 files belonging to 4 classes.


In [ ]:
data_augmentation_test=tf.keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
model.evaluate(test_dataset)
import pickle

22/22 [==============================] - 79s 2s/step - loss: 0.4553 - accuracy: 0.8905


In [ ]:
model.save('/content/gdrive/MyDrive/Classroom/my_model.h5')